# HW 3: Neural Machine Translation

In this homework you will build a full neural machine translation system using an attention-based encoder-decoder network to translate from German to English. The encoder-decoder network with attention forms the backbone of many current text generation systems. See [Neural Machine Translation and Sequence-to-sequence Models: A Tutorial](https://arxiv.org/pdf/1703.01619.pdf) for an excellent tutorial that also contains many modern advances.

## Goals


1. Build a non-attentional baseline model (pure seq2seq as in [ref](https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf)). 
2. Incorporate attention into the baseline model ([ref](https://arxiv.org/abs/1409.0473) but with dot-product attention as in class notes).
3. Implement beam search: review/tutorial [here](http://www.phontron.com/slides/nlp-programming-en-13-search.pdf)
4. Visualize the attention distribution for a few examples. 

Consult the papers provided for hyperparameters, and the course notes for formal definitions.

This will be the most time-consuming assignment in terms of difficulty/training time, so we recommend that you get started early!

## Setup

This notebook provides a working definition of the setup of the problem itself. Feel free to construct your models inline, or use an external setup (preferred) to build your system.

In [59]:
# Text text processing library and methods for pretrained word embeddings
from torchtext import data
from torchtext import datasets
import torch
from torchtext.vocab import Vectors, GloVe
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
from torch.optim.lr_scheduler import MultiStepLR
from matplotlib import pyplot as plt
from torch.nn.utils import clip_grad_norm
import numpy as np
import torch.nn.init as weight_init
import time
import os

We first need to process the raw data using a tokenizer. We are going to be using spacy, which can be installed via:  
  `[sudo] pip install spacy`  
  
Tokenizers for English/German can be installed via:  
  `[sudo] python -m spacy download en`  
  `[sudo] python -m spacy download de`
  
This isn't *strictly* necessary, and you can use your own tokenization rules if you prefer (e.g. a simple `split()` in addition to some rules to acccount for punctuation), but we recommend sticking to the above.

In [2]:
import spacy
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]


Note that we need to add the beginning-of-sentence token `<s>` and the end-of-sentence token `</s>` to the 
target so we know when to begin/end translating. We do not need to do this on the source side.

In [3]:
BOS_WORD = '<s>'
EOS_WORD = '</s>'
DE = data.Field(tokenize=tokenize_de)
EN = data.Field(tokenize=tokenize_en, init_token = BOS_WORD, eos_token = EOS_WORD) # only target needs BOS/EOS

Let's download the data. This may take a few minutes.

**While this dataset of 200K sentence pairs is relatively small compared to others, it will still take some time to train. So we are going to be only working with sentences of length at most 20 for this homework. Please train only on this reduced dataset for this homework.**

In [4]:
MAX_LEN = 20
train, val, test = datasets.IWSLT.splits(exts=('.de', '.en'), fields=(DE, EN), 
                                         filter_pred=lambda x: len(vars(x)['src']) <= MAX_LEN and 
                                         len(vars(x)['trg']) <= MAX_LEN)
print(train.fields)
print(len(train))
print(vars(train[0]))

{'trg': <torchtext.data.field.Field object at 0x7f39995032b0>, 'src': <torchtext.data.field.Field object at 0x7f39a7966a20>}
119076
{'trg': ['David', 'Gallo', ':', 'This', 'is', 'Bill', 'Lange', '.', 'I', "'m", 'Dave', 'Gallo', '.'], 'src': ['David', 'Gallo', ':', 'Das', 'ist', 'Bill', 'Lange', '.', 'Ich', 'bin', 'Dave', 'Gallo', '.']}


Now we build the vocabulary and convert the text corpus into indices. We are going to be replacing tokens that occurred less than 5 times with `<unk>` tokens, and take the rest as our vocab.

In [5]:
MIN_FREQ = 5
DE.build_vocab(train.src, min_freq=MIN_FREQ)
EN.build_vocab(train.trg, min_freq=MIN_FREQ)
print(DE.vocab.freqs.most_common(10))
print("Size of German vocab", len(DE.vocab))
print(EN.vocab.freqs.most_common(10))
print("Size of English vocab", len(EN.vocab))
print(EN.vocab.stoi["<s>"], EN.vocab.stoi["</s>"]) #vocab index for <s>, </s>

[('.', 113253), (',', 67237), ('ist', 24189), ('die', 23778), ('das', 17102), ('der', 15727), ('und', 15622), ('Sie', 15085), ('es', 13197), ('ich', 12946)]
Size of German vocab 13353
[('.', 113433), (',', 59512), ('the', 46029), ('to', 29177), ('a', 27548), ('of', 26794), ('I', 24887), ('is', 21775), ("'s", 20630), ('that', 19814)]
Size of English vocab 11560
2 3


Now we split our data into batches as usual. Batching for MT is slightly tricky because source/target will be of different lengths. Fortunately, `torchtext` lets you do this by allowing you to pass in a `sort_key` function. This will minimizing the amount of padding on the source side, but since there is still some padding you will inadvertendly "attend" to these padding tokens. 

One way to get rid of padding is to pass a binary `mask` vector to your attention module so its attention score (before the softmax) is minus infinity for the padding token. Another way (which is how we do it for our projects, e.g. opennmt) is to manually sort data into batches so that each batch has exactly the same source length (this means that some batches will be less than the desired batch size, though).

However, for this homework padding won't matter too much, so it's fine to ignore it.

In [6]:
BATCH_SIZE = 32
train_iter, val_iter = data.BucketIterator.splits((train, val), batch_size=BATCH_SIZE, device=-1,
                                                  repeat=False, sort_key=lambda x: len(x.src))

Let's check to see that the BOS/EOS token is indeed appended to the target (English) sentence.

In [7]:
batch = next(iter(train_iter))
print("Source")
print(batch.src)
print("Target")
print(batch.trg)


Source
Variable containing:

Columns 0 to 10 
    39   1126      8     12    241    277     12     12     40     28     73
   115    267     86      5   2081    101      0     37     48      0    751
   198      0   1124   1691     22    823   3038    126   2486     10     13
  1333     24   4178      4      0    445   4828    115     15      3     15
     3    381     29    226     36    116     33    368      0      8     30
   163      0     62     36      5     10     19      3    542     11   6787
    10   3286    275     46    946     52     19     10      5     67     78
   118   1126    423     22      8     22   9826      4     49     22      5
  7300    127   1567   3823   1626   3171      0      7   2591    324     21
     4    507      2    259   7558    829    729      0     54      0    852
     2      2    315      2      2      2      2     16      2      2      2

Columns 11 to 21 
    12    148     28     76    393     12     20     77    102     12     23
    11     

Success! Now that we've processed the data, we are ready to begin modeling.

## Assignment

Now it is your turn to build the models described at the top of the assignment. 

When a model is trained, use the following test function to produce predictions, and then upload to the kaggle competition: https://www.kaggle.com/c/cs287-hw3-s18/

For the final Kaggle test, we will provide the source sentence, and you are to predict the **first three words of the target sentence**. The source sentence can be found under `source_test.txt`

In [9]:
!head source_test.txt

head: cannot open 'source_test.txt' for reading: No such file or directory


Similar to HW1, you are to predict the 100 most probable 3-gram that will begin the target sentence. The submission format will be as follows, where each word in the 3-gram will be separated by "|", and each 3-gram will be separated by space. For example, here is what an example submission might look like with 5 most-likely 3-grams (instead of 100).

```
id,word
1,Newspapers|talk|about When|I|was Researchers|call|the Twentysomethings|like|Alex But|before|long
2,That|'s|what Newspapers|talk|about You|have|robbed It|'s|realizing My|parents|wanted
3,We|forget|how We|think|about Proust|actually|links Does|any|other This|is|something
4,But|what|do And|it|'s They|'re|on My|name|is It|only|happens
```

When you print out your data, you will need to escape quotes and commas with the following command so that Kaggle does not complain. 

In [54]:
use_cuda = torch.cuda.is_available()

In [64]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seqs, target_seqs):
        batch_size = input_seqs.size(1)
        target_length = target_seqs.size(0)
        vocab_size = self.decoder.output_size
        if use_cuda:
            outputs = Variable(torch.zeros(target_length, batch_size, vocab_size)).cuda()
        else:
            outputs = Variable(torch.zeros(target_length, batch_size, vocab_size))
        encoder_output, hidden = self.encoder(input_seqs, None)
        output = Variable(target_seqs.data[0, :]) 
        for t in range(1, target_length):
            output, hidden = self.decoder(output, hidden)
            outputs[t] = output
            #best = output[0].data.max(1)[1]
            if use_cuda:
                output = Variable(target_seqs.data[t]).cuda()
                #output = Variable(best).cuda()
            else:
                output = Variable(target_seqs.data[t])
                #output = Variable(best)
        return outputs
    
    def batch_train(self, optimizer, train_iter, vocab_size, grad_clip=10):
        self.train()
        total_loss = 0
        pad = EN.vocab.stoi['<pad>']
        curr_time = time.time()
        loss_f = nn.NLLLoss()
        for b, batch in enumerate(train_iter):
            source = batch.src
            target = batch.trg
            if use_cuda:
                source, target = source.cuda(), target.cuda()
            optimizer.zero_grad()
            output = self.forward(source, target)
            loss = F.cross_entropy(output[1:].view(-1, vocab_size),
                                   target[1:].contiguous().view(-1),
                                   ignore_index=pad)
            loss.backward()
            clip_grad_norm(self.parameters(), grad_clip)
            optimizer.step()
            total_loss += loss.data[0]

            if b % 1000 == 0 and b != 0:
                total_loss = total_loss / 1000
                print("[%d][loss:%5.2f][pp:%5.2f][time:%5.2f]" %
                      (b, total_loss, math.exp(total_loss), time.time() - curr_time))
                total_loss = 0
                curr_time = time.time()
                
                
    def predict(self, val_iter, vocab_size):
        self.eval()
        pad = EN.vocab.stoi['<pad>']
        total_loss = 0
        for batch in val_iter:
            source = batch.src
            target = batch.trg
            if use_cuda:
                source = Variable(source.data.cuda(), volatile=True)
                target = Variable(target.data.cuda(), volatile=True)
            output = self.forward(source, target)
            loss = F.cross_entropy(output[1:].view(-1, vocab_size),
                                   target[1:].contiguous().view(-1),
                                   ignore_index=pad)
            total_loss += loss.data[0]
        return total_loss / len(val_iter)

In [65]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size = 13352, hidden_size = 200, n_layers=2, dropout=0.3, bidirectional=None):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers, bidirectional=bidirectional)

    def forward(self, input, hidden):
        embedded = self.embedding(input)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden
        
class DecoderLSTM(nn.Module):
    def __init__(self, hidden_size=200, output_size=11560, n_layers = 2, dropout= 0.3, bidirectional=None):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = nn.Dropout(dropout, inplace=True)
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size, self.n_layers, bidirectional=bidirectional)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_seq, hidden):
        embedded = self.embedding(input_seq).unsqueeze(0)
        embedded = self.dropout(embedded)
        output, hidden = self.lstm(embedded, hidden)
        output = F.tanh(output)
        output = self.softmax(self.out(output))
        return output, hidden

In [66]:
encoder = EncoderLSTM(input_size = len(DE.vocab), hidden_size = 200, n_layers=2, dropout=0.3, bidirectional=None)
decoder = DecoderLSTM(hidden_size=200, output_size = len(EN.vocab), n_layers = 2, dropout= 0.3, bidirectional=None)
seq2seq = Seq2Seq(encoder, decoder).cuda()
epoch_num = 13
optimizer = optim.SGD(seq2seq.parameters(), lr=1)
scheduler = MultiStepLR(optimizer, milestones=range(9, epoch_num), gamma=0.5)

In [ ]:
best_val_loss = None
for i in range(epoch_num):
    seq2seq.batch_train(optimizer, train_iter,len(EN.vocab), grad_clip = 10)
    val_loss = seq2seq.predict(val_iter, len(EN.vocab))
    print("[Epoch:%d] val_loss:%5.3f | val_pp:%5.2fS"
          % (i, val_loss, math.exp(val_loss)))

    # Save the model if the validation loss is the best we've seen so far.
    if not best_val_loss or val_loss < best_val_loss:
        print("[!] saving model...")
        if not os.path.isdir(".save"):
            os.makedirs(".save")
        torch.save(seq2seq.state_dict(), './.save/seq2seq_%d.pt' % (i))
        best_val_loss = val_loss
test_loss = seq2seq.predict(test_iter, len(EN.vocab))
print("[TEST] loss:%5.2f" % test_loss)

In [147]:
def escape(l):
    return l.replace("\"", "<quote>").replace(",", "<comma>")

You should perform your hyperparameter search/early stopping/write-up based on perplexity, not the above metric. (In practice, people use a metric called [BLEU](https://www.aclweb.org/anthology/P02-1040.pdf), which is roughly a geometric average of 1-gram, 2-gram, 3-gram, 4-gram precision, with a brevity penalty for producing translations that are too short.)

Finally, as always please put up a (short) write-up following the template provided in the repository:  https://github.com/harvard-ml-courses/cs287-s18/blob/master/template/
